In [21]:
import pandas as pd
import pickle
import sqlite3

In [22]:
# UNPICKLE CITYBIKES DF
with open('../data/citybikes_filled_df.pickle', 'rb') as df:
    citybikes_df = pickle.load(df)

citybikes_df.head(5)

,name,id,timestamp,longitude,latitude,slots,free_bikes,empty_slots,has_ebikes,ebikes,payment,renting,returning
0,Harmon St at Adeline St,d0e8f4f1834b7b33a3faf8882f567ab8,2024-04-02T18:29:27.066000Z,-122.270582,37.849735,15,9,6,True,0,"[key, creditcard]",1,1
1,Fountain Alley at S 2nd St,983514094dd808b1604da2dcfc2d09af,2024-04-02T18:29:26.582000Z,-121.889277,37.336188,11,8,3,True,6,"[key, creditcard]",1,1
2,Oak St at 1st St,da17603652106fda93da4e255a5b0a22,2024-04-02T18:29:27.044000Z,-121.88109,37.322125,23,14,9,True,7,"[key, creditcard]",1,1
3,Bestor Art Park,7a21c92b3b4cd2f7759107b4fdebf869,2024-04-02T18:29:26.797000Z,-121.874119,37.323678,15,12,3,True,3,"[key, creditcard]",1,1
4,5th St at Virginia St,ce34d38fb230a23c1ced12d1e16df294,2024-04-02T18:29:26.971000Z,-121.87712,37.325998,27,19,7,True,12,"[key, creditcard]",1,1


In [4]:
# UNPICKLE FSQ DF 
with open('../data/fsq_places.pickle', 'rb') as df:
    fsq_places = pickle.load(df)

fsq_places.head(5)

,reference_bike_stn,fsq_id,cat_id,cat_name,lat,long,name,street_address,zip,locality,distance
0,d0e8f4f1834b7b33a3faf8882f567ab8,4b3fc82cf964a520a1ae25e3,13303,Mexican Restaurant,37.850052,-122.270523,Las Palmas Super Burritos and Heros,3221 Adeline St,94703,Berkeley,33
1,d0e8f4f1834b7b33a3faf8882f567ab8,57da76a603a44d3f9b3ddd3b,10004,Art Gallery,37.849798,-122.271728,Firehouse Collective East/West Art Studios,1835 Harmon St,94703,Berkeley,100
2,d0e8f4f1834b7b33a3faf8882f567ab8,4b64bd80f964a52074cc2ae3,13064,Pizzeria,37.849316,-122.271629,Domino's Pizza,3264 Adeline St,94703,Berkeley,113
3,d0e8f4f1834b7b33a3faf8882f567ab8,4c12e9a51b5cef3b9cd7edc4,13102,Cantonese Restaurant,37.848703,-122.270660,Chen's Garden,1818 Alcatraz Ave,94703,Berkeley,119
4,d0e8f4f1834b7b33a3faf8882f567ab8,52fe4532498e80c8d231d768,13034,Café,37.850958,-122.271077,Rasa Caffe,3140 Martin Luther King Jr Way,94703,Berkeley,141


In [3]:
# UNPICKLE YELP DF 
with open('../data/yelp_df.pickle', 'rb') as df:
    yelp_df = pickle.load(df)

yelp_df.head(5)

,reference_bike_stn,yelp_id,cat_alias,lat,long,name,street_address,zip,city,price,rating,review_count,distance_from_bike_stn
0,d0e8f4f1834b7b33a3faf8882f567ab8,OjMumhyL26n5fDZ2CQLnyg,ethiopian,37.850301,-122.271266,Lemat,3212 Adeline St,94703,Berkeley,$$,4.4,239,86.993559
1,d0e8f4f1834b7b33a3faf8882f567ab8,-EpmBwODsvQ6gHx2IuVO5w,cafes,37.850017,-122.270532,El Tiny Cafe,3219 Adeline St,94703,Berkeley,None,4.8,60,31.707767
2,d0e8f4f1834b7b33a3faf8882f567ab8,shRP4Vxu2hDdmdsHf-5DmQ,burgers,37.848520,-122.260380,A+ Burger,6228 Telegraph Ave,94609,Oakland,$$,4.4,859,895.542416
3,d0e8f4f1834b7b33a3faf8882f567ab8,pJdwr6JoCVXTKbdJlm-rdg,italian,37.850900,-122.278200,Creekwood,3121 Sacramento St,94702,Berkeley,$$,4.6,481,684.391900
4,d0e8f4f1834b7b33a3faf8882f567ab8,nt46Amrqx5qwTyNEcO54ew,burgers,37.843820,-122.276750,Sideshow Kitchen,942 Stanford Ave,94608,Oakland,$$,4.4,496,853.805062


# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

In [23]:
con = sqlite3.connect("../data/biking_bayarea.db") # create connection to database

In [11]:
column_names_list = citybikes_df.columns.tolist()
column_names_list

['name',
 'id',
 'timestamp',
 'longitude',
 'latitude',
 'slots',
 'free_bikes',
 'empty_slots',
 'has_ebikes',
 'ebikes',
 'payment',
 'renting',
 'returning']

In [24]:
cur = con.cursor() # create database cursor, used to execute SQL queries & fetch results

In [16]:
cur.execute("""
CREATE TABLE citybikes (
    name VARCHAR(50),
    id VARCHAR(100) PRIMARY KEY NOT NULL,
    timestamp TIMESTAMP,
    longitude VARCHAR(50),
    latitude VARCHAR(50),
    slots INT UNSIGNED,
    free_bikes INT UNSIGNED,
    empty_slots INT UNSIGNED,
    has_ebikes BOOLEAN,
    ebikes INT UNSIGNED,
    payment VARCHAR(50),
    renting INT UNSIGNED,
    is_returning INT UNSIGNED
);
""")

In [ ]:
cur.execute(f"""
INSERT INTO citybikes VALUES
    ({name}, {id}, {timestamp}, {long}, {lat}, {slots}, {free_bikes}, {empty_slots}, {has_ebikes}, {ebikes}, {payment}, {renting}, {is_returning})
""")

In [18]:
for index, row in citybikes_df.iterrows():
    name = row['name']
    bike_id = row['id']
    timestamp = row['timestamp']
    longitude = row['longitude']
    latitude = row['latitude']
    slots = row['slots']
    free_bikes = row['free_bikes']
    empty_slots = row['empty_slots']
    has_ebikes = row['has_ebikes']
    ebikes = row['ebikes']
    payment = row['payment']
    renting = row['renting']
    is_returning = row['returning']



SyntaxError: incomplete input (3238102454.py, line 2)

In [27]:
def populate_citybikes_sqllite(con, cur, citybikes_df):
    for index, row in citybikes_df.iterrows():
        name = row['name']
        bike_id = row['id']
        timestamp = row['timestamp']
        long = row['longitude']
        lat = row['latitude']
        slots = row['slots']
        free_bikes = row['free_bikes']
        empty_slots = row['empty_slots']
        has_ebikes = row['has_ebikes']
        ebikes = row['ebikes']
        payment = row['payment']
        renting = row['renting']
        is_returning = row['returning']
        
        cur.execute(f"""
            INSERT INTO citybikes VALUES
                ({name}, {id}, {timestamp}, {long}, {lat}, {slots}, {free_bikes}, {empty_slots}, {has_ebikes}, {ebikes}, {payment}, {renting}, {is_returning});
        """)

        con.commit()
        

In [28]:
populate_citybikes_sqllite(con, cur, citybikes_df)

OperationalError: near "St": syntax error

In [6]:
con.close()

Provide a visualization that you used as part of your EDA process. Explain the initial pattern or relationship you discoved through this visualization. 

Look at the data before and after the join to validate your data.